# EDA & Insights – Winter Mountain Tour Demand & Cancellations

## Objectives
- Explore seasonal booking patterns by region.
- Quantify the uplift from bank holiday weeks and peak winter.
- Understand how weather and calendar features relate to bookings.
- Explore cancellation patterns, especially under severe weather.
- Generate figures and narrative insights to support hypotheses H1–H3.

## Inputs
- `data/processed/weekly_bookings_regression.csv`
- `data/processed/bookings_for_classification.csv`

## Outputs
- EDA figures saved to `reports/figures/`
- Textual insights that will be re-used in the README and dashboard.


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

BASE_DIR = Path("..").resolve()
DATA_PROCESSED = BASE_DIR / "data" / "processed"
FIG_DIR = BASE_DIR / "reports" / "figures"

FIG_DIR.mkdir(parents=True, exist_ok=True)

sns.set(style="whitegrid")


In [ ]:
weekly = pd.read_csv(
    DATA_PROCESSED / "weekly_bookings_regression.csv",
    parse_dates=["week_start"]
)

bookings_clf = pd.read_csv(
    DATA_PROCESSED / "bookings_for_classification.csv",
    parse_dates=["tour_date", "booking_date", "week_start"]
)

weekly.head(), bookings_clf.head()


In [ ]:
plt.figure(figsize=(14, 6))
for region, grp in weekly.groupby("region"):
    plt.plot(grp["week_start"], grp["bookings_count"], label=region)

plt.title("Weekly Bookings Over Time by Region")
plt.xlabel("Week start")
plt.ylabel("Bookings count")
plt.legend()
plt.tight_layout()

fig_path = FIG_DIR / "weekly_bookings_by_region.png"
plt.savefig(fig_path, dpi=120)
fig_path


### Insight 1 – Seasonality Patterns by Region (supports H1, H3)

The weekly bookings time series shows clear **seasonal peaks** during winter months
and known holiday periods. Regions such as the Lake District exhibit consistently
higher baseline demand compared to others, with visible surges during peak weeks.

This supports the idea that:
- Seasonality and holidays are key drivers of demand (H1).
- Recent history (lagged demand) is likely to be informative, as peaks and troughs
  tend to persist over short time windows (H3).


In [ ]:
# Use year-week as a string label for plotting
weekly["year_week"] = weekly["week_start"].dt.strftime("%Y-%U")

pivot = weekly.pivot_table(
    index="year_week",
    columns="region",
    values="bookings_count",
    aggfunc="sum"
)

plt.figure(figsize=(10, 10))
sns.heatmap(pivot, cmap="viridis")
plt.title("Heatmap of Weekly Bookings by Region")
plt.xlabel("Region")
plt.ylabel("Year-Week")
plt.tight_layout()

fig_path = FIG_DIR / "heatmap_week_region_bookings.png"
plt.savefig(fig_path, dpi=120)
fig_path


### Insight 2 – Week × Region Heatmap

The heatmap highlights **clusters of high-demand weeks** across regions.
Peak-intensity bands align with winter and holiday periods, confirming that
these time windows consistently generate more bookings.

This reinforces:
- The importance of modelling **calendar features** (week number, month,
  peak winter, bank holiday flags).
- The need for the dashboard to show differences across regions so
  Operations can plan region-specific staffing.


In [ ]:
holiday_summary = (
    weekly.groupby("is_bank_holiday_week", as_index=False)["bookings_count"]
    .mean()
    .rename(columns={"bookings_count": "avg_bookings"})
)

holiday_summary["holiday_label"] = holiday_summary["is_bank_holiday_week"].map(
    {0: "Non-holiday weeks", 1: "Bank holiday weeks"}
)

holiday_summary

plt.figure(figsize=(6, 4))
sns.barplot(
    data=holiday_summary,
    x="holiday_label",
    y="avg_bookings"
)
plt.title("Average Weekly Bookings – Holiday vs Non-holiday Weeks")
plt.xlabel("")
plt.ylabel("Average bookings")
plt.tight_layout()

fig_path = FIG_DIR / "holiday_uplift_bar.png"
plt.savefig(fig_path, dpi=120)
fig_path



### Insight 3 – Holiday Uplift in Weekly Bookings (H1)

The bar chart shows that **bank holiday weeks have higher average bookings**
than non-holiday weeks. This supports **Hypothesis H1**: holidays create
a measurable uplift in demand across the regions.

Operationally, this means:
- Bank holiday weeks should be treated as **high-priority weeks** for staffing and capacity planning.
- The regression model should include bank holiday flags as key features to capture this uplift.
